<a href="https://colab.research.google.com/github/jsum007/Coursera_DL_Specialization/blob/master/cs772_assg1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nltk
!pip install autocorrect
import nltk
nltk.download('gutenberg')
nltk.download('punkt')
from nltk.corpus import gutenberg
import numpy as np

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#Dataset Creation

In [ ]:
book_names = gutenberg.fileids()
sentences_list = []
for i in range(len(book_names)):
  book_sents = gutenberg.sents(book_names[i])
  sentences_list+= list(book_sents)
print(len(sentences_list))


98503


In [ ]:
from nltk import tokenize
import re
with open('Analogy_dataset.txt', 'r') as f:
  analogy_words = f.read().split()
analogy_words = [*set(analogy_words)]
#print(len(analogy_lines))

num_sent_max = 100

import requests
from bs4 import BeautifulSoup

for i in range(len(analogy_words)):
  wordi = analogy_words[i]
  # Making a GET request
  r = requests.get('https://en.wikipedia.org/wiki/' + wordi)
  # Parsing the HTML
  soup = BeautifulSoup(r.content, 'html.parser')
  #print(soup.prettify())
  ss = soup.find_all('div', class_ ='mw-parser-output')
  counter = 0
  indic = 0
  for s in ss:
    content = s.find_all('p')

    for line in content:
      thisline = line.text
      thissents = tokenize.sent_tokenize(thisline)
      for sentence in thissents: #checking all sentences of a particular wiki page
        if wordi in sentence:
          while not sentence[0].isalpha():  #to get a clean sentence without [21] etc from wikipedia.
            sentence = sentence[1:]
          counter = counter+1
          sentences_list.append(sentence)
          sentence = "".join(re.split("\(|\)|\[|\]", sentence)[::2])
          print(sentence)
          if counter > num_sent_max:
            indic=1
            break
      if indic == 1:
        break

Streaming output truncated to the last 5000 lines.
Budapest is among the 25 most visited cities in the world, the city welcoming more than 4.4 million international visitors each year, therefore the traditional and the congress tourism industry also deserve a mention, it contributes greatly to the city's economy.
Budapest Stock Exchange, key institution of the publicly offered securities in Hungary and Central and Eastern Europe is situated in Budapest's CBD at Liberty Square.
Large Hungarian multinational corporations headquartered in Budapest are listed on BSE, for instance the Fortune Global 500 firm MOL Group, the OTP Bank, FHB Bank, Gedeon Richter, Magyar Telekom, CIG Pannonia, Zwack Unicum and more.
Nowadays nearly all branches of industry can be found in Budapest, there is no particularly special industry in the city's economy, but the financial centre role of the city is strong, nearly 40 major banks are presented in the city, also those like Bank of China, KDB Bank and Hanwha 

In [ ]:
len(sentences_list)

108800

Data Pre-processing

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import words
from autocorrect import Speller

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('words')
nltk.download('omw-1.4')
  

stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()
english_words = set(words.words())

spell = Speller()

def preprocess_sentences(sentences):
    preprocessed_sentences = []
    for words in sentences:
        # words = word_tokenize(sentence)
        # words = [spell(word) for word in words]
        words = [word.lower() for word in words]
        words = [lemmatizer.lemmatize(word) for word in words if word.isalpha() and word not in stop_words and word in english_words]
        words = list(set(words))
        preprocessed_sentence = " ".join(words)
        preprocessed_sentences.append(preprocessed_sentence)
    return list(set(preprocessed_sentences))

# sentences = [
#     "This is a sentence for spell corection.",
#     "Another sentence for spell correctiong",
#     "Stop words are removed in this sentence",
#     "This sentence is lemmatized",
#     "And this sentence is deduplicated"
# ]

# preprocessed_sentences = preprocess_sentences(sentences)
# print(preprocessed_sentences)
processed_sentences = preprocess_sentences(sentences_list)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
processed_sentences

['',
 'humble leave deprecation thus lord universal speech',
 'selah ye god earth sing lord unto',
 'sense antique clerical necktie except brow gave antiquity say strictly across cut barred looking exactly white could',
 'husband stain trough happy bedstead work lie day friendly roof woman little floor supper well shroud married bride bed shape cradle home content jack family babe coffin cook saw wife eaten beneath within joyously man join chaste dead young got',
 'behold mine let life deliver much tribulation thy lord day set',
 'chamber forth north toward brought way building utter place court separate',
 'well sham honesty courage',
 'god thee men said unto came',
 'cried scornfully',
 'friend leave',
 'thus displeased whereto fatherly',
 'u n h c e r',
 'born man something ever woman',
 'black towards front slipping loose coat figure house',
 'comfort heart thine early morning fifth pray depart thee day said damsel arose father',
 'hat',
 'hair thought',
 'went covered mouth king w

In [ ]:

from tqdm import tqdm

# activation function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# gradient
def sigmoid_derivative(out):
    return out * (1 - out)

# Define the training function for the CBOW model
def train_cbow(sentences_list, learning_rate=0.1, epochs=100):
    # Initialize the word vectors for each word in the vocabulary
    vocab = set()
    for sentence in sentences_list:
        for word in sentence.split():
            vocab.add(word)
    vocab = list(vocab)
    print(vocab)
    vocab_size = len(vocab)
    l2_neurons = 300
    



    # Context size for neighbouring words in sentence
    context = 2

    # Initialize the weight matrices for the hidden and output layers
    W1 = np.random.randn(vocab_size, l2_neurons)
    W2 = np.random.randn(l2_neurons, vocab_size)

    for epoch in range(epochs):
        # Loop through each sentence in the training data
        for sentence in tqdm(sentences_list):

            # Initialize the hidden layer activations
            hidden_layer = np.zeros((1, l2_neurons))
            words = sentence.split()
            # Loop through each word in the sentence
            for i, word in enumerate(words):
                # Look up the word vector for the current word
                list_of_onehot_vecs = []
                j = 1
                while(i-j>=0 and j<context):
                    target_word_vector = np.zeros((1, vocab_size))
                    target_word_vector[0, vocab.index(words[i - j])] = 1
                    list_of_onehot_vecs.append(target_word_vector)
                    j +=1
                j = 1
                while(i +j < len(words) and j<context):
                    target_word_vector = np.zeros((1, vocab_size))
                    target_word_vector[0, vocab.index(words[i + j])] = 1
                    list_of_onehot_vecs.append(target_word_vector) 
                    j +=1

                word_input_vector = np.mean(list_of_onehot_vecs, axis = 0)

                # Compute the input to the hidden layer
                hidden_layer_input = np.dot(word_input_vector, W1)

                # Compute the activations in the hidden layer
                hidden_layer = sigmoid(hidden_layer_input)

                # Compute the input to the output layer
                output_layer_input = np.dot(hidden_layer, W2)

                # Compute the activations in the output layer
                output_layer = sigmoid(output_layer_input)

                # Compute the target word vector
                target_word_vector = np.zeros((1, vocab_size))
                target_word_vector[0, vocab.index(words[i])] = 1
                

                # Compute the error in the output layer
                error = target_word_vector - output_layer

                # Compute the gradients in the output layer
                output_layer_delta = -error * sigmoid_derivative(output_layer)

                # Compute the error in the hidden layer
                hidden_layer_error = np.dot(output_layer_delta, W2.T)

                # Compute the gradients in the hidden layer
                hidden_layer_delta = hidden_layer_error * sigmoid_derivative(hidden_layer)

                # Update the weights
                W2 -= learning_rate * np.dot(hidden_layer.T, output_layer_delta)
                W1 -= learning_rate * np.dot(word_input_vector.T, hidden_layer_delta)


    # Return the trained word vectors and weight matrices
    return W1, W2

In [ ]:

from tqdm import tqdm

# activation function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# gradient
def sigmoid_derivative(out):
    return out * (1 - out)

# Define the training function for the CBOW model
def train_cbow(sentences_list, learning_rate=0.1, epochs=100):
    # Initialize the word vectors for each word in the vocabulary
    vocab = set()
    for sentence in sentences_list:
        for word in sentence.split():
            vocab.add(word)
    vocab = list(vocab)
    print(vocab)
    vocab_size = len(vocab)
    l2_neurons = 300
    



    # Context size for neighbouring words in sentence
    context = 2

    # Initialize the weight matrices for the hidden and output layers
    W1 = np.random.randn(vocab_size, l2_neurons)
    W2 = np.random.randn(l2_neurons, vocab_size)

    batch_size = 200
    numtotal = 0

    for epoch in range(epochs):
        total_batch_vecsin=[]
        total_batch_vecsout=[]
        
        # Loop through each sentence in the training data
        for sentence in tqdm(sentences_list):
          numtotal = numtotal+1
          
          hidden_layer = np.zeros((1, l2_neurons))
          words = sentence.split()
          # Loop through each word in the sentence
          list_of_onehot_vecs = []

          for i, word in enumerate(words):
              # Look up the word vector for the current word
              j = 1
              while(i-j>=0 and j<context):
                  target_word_vector = np.zeros((1, vocab_size))
                  target_word_vector[0, vocab.index(words[i - j])] = 1
                  list_of_onehot_vecs.append(target_word_vector)
                  j +=1
              j = 1
              while(i +j < len(words) and j<context):
                  target_word_vector = np.zeros((1, vocab_size))
                  target_word_vector[0, vocab.index(words[i + j])] = 1
                  list_of_onehot_vecs.append(target_word_vector) 
                  j +=1
              
              if(len(list_of_onehot_vecs)==0):
                  continue
              word_input_vector = np.mean(list_of_onehot_vecs, axis = 0)
              #print("pre_vect",word_input_vector.shape)
              total_batch_vecsin.append(word_input_vector)
              target_word_vector = np.zeros((1, vocab_size))
              target_word_vector[0, vocab.index(words[i])] = 1
              total_batch_vecsout.append(target_word_vector)
              #print("total_batch",np.array(total_batch_vecsin).shape)

          if numtotal%batch_size == 0:
            # Initialize the hidden layer activations
                word_input_vector = np.array(total_batch_vecsin)
                target_word_vector = np.array(total_batch_vecsout)

                total_batch_vecsin=[]
                total_batch_vecsout=[]

                #print("word_vect",word_input_vector.shape)

                # Compute the input to the hidden layer
                hidden_layer_input = np.dot(word_input_vector, W1)

                # Compute the activations in the hidden layer
                hidden_layer = sigmoid(hidden_layer_input)

                # Compute the input to the output layer
                output_layer_input = np.dot(hidden_layer, W2)

                # Compute the activations in the output layer
                output_layer = sigmoid(output_layer_input)

                # Compute the error in the output layer
                error = target_word_vector - output_layer

                # Compute the gradients in the output layer
                output_layer_delta = -error * sigmoid_derivative(output_layer)

                # Compute the error in the hidden layer
                hidden_layer_error = np.dot(output_layer_delta, W2.T)

                # Compute the gradients in the hidden layer
                hidden_layer_delta = hidden_layer_error * sigmoid_derivative(hidden_layer)

                # Update the weights
                print("out 1 shape",np.dot(hidden_layer, output_layer_delta.T).shape)
                print("out 2 shape",np.dot(word_input_vector, hidden_layer_delta.T).shape)

                W2 -= learning_rate * np.sum(np.dot(hidden_layer.T, output_layer_delta.T), axis = 0)
                W1 -= learning_rate * np.sum(np.dot(word_input_vector.T, hidden_layer_delta.T), axis = 0)

        break

    # Return the trained word vectors and weight matrices
    return W1, W2

In [ ]:
import numpy as np
a = [np.array([0,1,0]), np.array([1,0,0])]
print(np.mean(a, axis = 0))

[0.5 0.5 0. ]


In [ ]:
W1, W2 = train_cbow(processed_sentences, learning_rate=0.1, epochs=5)
processed_sentences[0]

with open('cbow.npy', 'wb') as f:
    np.save(f, W1)
    np.save(f, W2)

['raging', 'alike', 'correspond', 'knave', 'dissatisfied', 'scall', 'practical', 'crumb', 'sprain', 'indelicacy', 'amuck', 'whiteness', 'wer', 'gratitude', 'restricted', 'screen', 'dissect', 'untended', 'breeze', 'durability', 'intoxicate', 'dizzy', 'respecter', 'hat', 'free', 'bitterness', 'accurately', 'inhuman', 'illustrative', 'pretended', 'sluice', 'participate', 'noisy', 'gaily', 'accrue', 'dredging', 'inflate', 'fetor', 'bombardment', 'viewless', 'wilily', 'conversation', 'food', 'sailer', 'weighty', 'physiologist', 'uneasy', 'unseduced', 'terribly', 'supplement', 'appallingly', 'dextrously', 'witching', 'stimulus', 'nun', 'tattooing', 'privileged', 'sling', 'commend', 'sophy', 'intentional', 'contest', 'bane', 'prodigy', 'curbing', 'navigator', 'considering', 'presage', 'tempter', 'unbounded', 'bramble', 'flooding', 'university', 'seizure', 'remedy', 'morality', 'thankless', 'proportionately', 'reputation', 'crisscross', 'pol', 'stagnant', 'disgraceful', 'grosser', 'haft', 'fou

  0%|          | 196/88253 [00:13<33:04, 44.36it/s]

In [ ]:
with open('cbow.npy', 'rb') as f:
    word_vec = np.load(f)
    W1 = np.load(f)
    W2 = np.load(f)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

The vocabulary size is-

In [ ]:
len(word_vec)

Analogy Operation

In [ ]:
# Skip-gram
def train_skipgram(sentences_list, learning_rate=0.1, epochs=10000):
    # Initialize the word vectors for each word in the vocabulary
    vocab = set()
    for sentence in sentences_list:
        for word in sentence.split():
            vocab.add(word)
    vocab = list(vocab)
    print(vocab)
    vocab_size = len(vocab)
    l2_neurons = 300

    # Context size for neighbouring words in sentence
    context = 2

    # Initialize the weight matrices for the hidden and output layers
    W1 = np.random.randn(vocab_size, l2_neurons)
    W2 = np.random.randn(l2_neurons, vocab_size)

    for epoch in range(epochs):
        # Loop through each sentence in the training data
        for sentence in tqdm(sentences_list):

            # Initialize the hidden layer activations
            hidden_layer = np.zeros((1, l2_neurons))
            words = sentence.split()
            # Loop through each word in the sentence
            for i, word in enumerate(words):
                # Look up the word vector for the current word
                list_of_onehot_vecs = []
                j = 1
                while(i-j>=0 and j<context):
                    target_word_vector = np.zeros((1, vocab_size))
                    target_word_vector[0, vocab.index(words[i - j])] = 1
                    list_of_onehot_vecs.append(target_word_vector)
                    j +=1
                j = 1
                while(i +j < len(words) and j<context):
                    target_word_vector = np.zeros((1, vocab_size))
                    target_word_vector[0, vocab.index(words[i + j])] = 1
                    list_of_onehot_vecs.append(target_word_vector) 
                    j +=1

                word_input_vector = np.zeros((1, vocab_size))
                word_input_vector[0, vocab.index(words[i])] = 1
                
                
                for k in range(len(list_of_onehot_vecs)):
                  # Compute the input to the hidden layer
                  hidden_layer_input = np.dot(word_input_vector, W1)
                  hidden_layer = sigmoid(hidden_layer_input)
                  output_layer_input = np.dot(hidden_layer, W2)
                  output_layer = sigmoid(output_layer_input)

                  target_word_vector = list_of_onehot_vecs[k]

                  # Compute the error in the output layer
                  error = target_word_vector - output_layer
                  # Compute the gradients in the output layer
                  output_layer_delta = -error * sigmoid_derivative(output_layer)
                  # Compute the error in the hidden layer
                  hidden_layer_error = np.dot(output_layer_delta, W2.T)
                  # Compute the gradients in the hidden layer
                  hidden_layer_delta = hidden_layer_error * sigmoid_derivative(hidden_layer)
                  # Update the weights
                  W2 -= learning_rate * np.dot(hidden_layer.T, output_layer_delta)
                  W1 -= learning_rate * np.dot(word_input_vector.T, hidden_layer_delta)


    # Return the trained word vectors and weight matrices
    return W1, W2


In [ ]:
W1, W2 = train_skipgram(processed_sentences, learning_rate=0.1, epochs=5)
processed_sentences[0]


['partiality', 'voice', 'reflecting', 'nipping', 'hence', 'tendency', 'distend', 'fix', 'deceivableness', 'feignedly', 'scarf', 'inbred', 'arrogantly', 'jackass', 'occasion', 'afoul', 'agnostically', 'fracas', 'kindhearted', 'flexible', 'unhurried', 'lavish', 'definitely', 'stated', 'devotion', 'tane', 'meeting', 'intermeddling', 'rase', 'cove', 'consolidation', 'preamble', 'administration', 'giver', 'afford', 'candidly', 'wobbling', 'woven', 'glorious', 'research', 'bitterness', 'choral', 'purgation', 'breasting', 'idleness', 'equator', 'myriad', 'reception', 'bleed', 'bewilderment', 'setter', 'swerve', 'ducking', 'foul', 'purtenance', 'scientific', 'hoof', 'enforce', 'session', 'tip', 'oscillating', 'bowl', 'ordination', 'plainness', 'boule', 'advise', 'charming', 'pestilence', 'misery', 'lived', 'lump', 'popularity', 'heighten', 'audibly', 'remembrancer', 'adjust', 'lamplight', 'precise', 'keel', 'fusion', 'looseness', 'doubly', 'maudlin', 'discontent', 'pose', 'unfaltering', 'retin

  0%|          | 143/85827 [03:11<31:47:37,  1.34s/it]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-12-7ebb4ac79469>", line 1, in <module>
    W1, W2 = train_skipgram(processed_sentences, learning_rate=0.1, epochs=5)
  File "<ipython-input-11-e50cc97b1326>", line 66, in train_skipgram
    W2 -= learning_rate * np.dot(hidden_layer.T, output_layer_delta)
  File "<__array_function__ internals>", line 5, in dot
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py", line 2040, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/loc

KeyboardInterrupt: ignored